对土地利用斑块数进行分类保存

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
from rasterio.enums import Resampling
import dask.array as da
import cv2
from skimage.util import view_as_blocks
import gc
import os

In [2]:
shape_frame = gpd.read_file(
    "/home/zyzh0/coding/risk_evaluation/scripts/datas/BOARDER/extracted/华北平原地市级/华北平原地市级.shp")
boundary = shape_frame.dissolve()


In [3]:
clcd_frame = xr.open_dataset("/home/zyzh0/coding/risk_evaluation/scripts/outputs/CLCD/CLCD_v01_2015.nc",decode_coords="all")
clcd_frame.rio.write_crs("epsg:4326", inplace=True)
clcd_frame.rio.write_transform(inplace=True)

<xarray.Dataset>
Dimensions:                        (y: 55661, x: 48240)
Coordinates:
  * y                              (y) float64 43.0 43.0 43.0 ... 28.0 28.0 28.0
  * x                              (x) float64 110.0 110.0 110.0 ... 123.0 123.0
    spatial_ref                    int64 0
Data variables:
    __xarray_dataarray_variable__  (y, x) float64 ...

In [4]:
clcd_frame_sliced = clcd_frame.loc[dict(y=slice(43,28.000025))]

In [5]:
clcd_frame_sliced.rio.write_transform(inplace=True)

<xarray.Dataset>
Dimensions:                        (y: 55660, x: 48240)
Coordinates:
  * y                              (y) float64 43.0 43.0 43.0 ... 28.0 28.0 28.0
  * x                              (x) float64 110.0 110.0 110.0 ... 123.0 123.0
    spatial_ref                    int64 0
Data variables:
    __xarray_dataarray_variable__  (y, x) float64 ...

In [6]:
clcd_array = clcd_frame_sliced.__xarray_dataarray_variable__.values

In [7]:
clcd_array.shape

(55660, 48240)

In [8]:
clcd_array_block = view_as_blocks(clcd_array, block_shape=(23,24))

In [9]:
clcd_ma_isin = np.isin(clcd_array_block,1)

In [10]:
clcd_ma_isin.shape

(2420, 2010, 23, 24)

In [11]:
clcd_ma_isin[:,:,1,1].shape

(2420, 2010)

In [22]:
cv2.connectedComponentsWithStats(clcd_ma_isin[1,1,:,:].astype(np.int8),4,cv2.CV_32S)[0]

1

In [2]:
def mask_dataset(dataset: xr.Dataset, mask_type: int) -> xr.Dataset:
    dataset_masked = dataset.where(dataset == mask_type)
    return dataset_masked

def reproject_original_dataset(dataset: xr.Dataset, shape: tuple, resampling: Resampling = Resampling.nearest) -> xr.Dataset:
    dataset.rio.write_crs("epsg:4326", inplace=True)
    dataset_reprojected = dataset.rio.reproject(
        dataset.rio.crs, shape=shape, resampling=resampling)
    return dataset_reprojected

def get_array_from_dataset(dataset: xr.Dataset) -> np.ndarray:
    array =  np.nan_to_num(dataset.__xarray_dataarray_variable__.values.astype(np.uint8),copy=False)
    return array

def patch_array(array: np.ndarray, patch_size: tuple) -> np.ndarray:
    array_patched = view_as_blocks(array, block_shape=patch_size)
    return array_patched

def get_N_number(types_array:np.ndarray) -> int:
    if types_array.all():
        return 1
    out = cv2.connectedComponentsWithStats(types_array, 4, cv2.CV_32S)
    return out[0]-1

In [3]:
def calc_patch_number_matrix_and_save(basedir:str,year:int,mask_type:int,patched_array:np.ndarray):
    #clcd_array = get_array_from_dataset(frame_array)
    #patched_clcd_array = patch_array(clcd_array, patch_size=(24,24))
    #del clcd_masked , clcd_reprojected , clcd_array
    #gc.collect()
    #print("patched")

    test_N_number_array = np.zeros_like(patched_array[:,:,1,1])

    test_area_of_types = np.zeros_like(test_N_number_array)

    def calc(i,j) : test_N_number_array[i,j] = get_N_number(patched_array[i,j])#; test_area_of_types[i,j] = np.sum(patched_array[i,j])

    [[calc(i,j) for i in range(patched_array.shape[0])] for j in range(patched_array.shape[1])]

    np.save(os.path.join(basedir,str(year)+"_"+str(mask_type)),test_N_number_array)

    np.save(os.path.join("/home/zyzh0/coding/risk_evaluation/scripts/outputs/CLCD_AREA_OF_TYPES",str(year)+"_"+str(mask_type)+"_area"),test_area_of_types)

    del test_N_number_array, test_area_of_types
    gc.collect()

    return mask_type

In [ ]:
for i in range(2000,2016):
    filepath = "/home/zyzh0/coding/risk_evaluation/scripts/outputs/CLCD/CLCD_v01_"+str(i)+".nc"
    print("working on "+str(i))
    clcd_frame = xr.open_dataset(filepath)
    clcd_frame = clcd_frame.loc[dict(y=slice(43,28.000025))]
    print("sliced")
    clcd_frame.rio.write_crs("epsg:4326", inplace=True)
    clcd_frame.rio.write_transform(inplace=True)
    frame_array = get_array_from_dataset(clcd_frame)
    patched_array = patch_array(frame_array, patch_size=(23,24))
    print("patched")
    for j in range(1,10):
        mask_type = j
        array_masked = np.isin(patched_array,mask_type).astype(np.uint8)
        print("masked type "+str(j))
        calc_patch_number_matrix_and_save("/home/zyzh0/coding/risk_evaluation/scripts/outputs/CLCD_N_NUMBER",i,j,array_masked)
        print("finished "+str(i)+"_"+str(j))
    del clcd_frame
    gc.collect()